In [14]:
import requests
r = requests.get("https://example.com")
print(r.status_code)

import sys
import os
# install package direcly in notebook: %pip install requests-html
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path

200


In [11]:
import pandas as pd
from collections import Counter
import re

def most_frequent_terms(df, column, top_n=10):
    """
    Return the most frequent terms from a text column in a DataFrame.
    
    Args:
        df (pd.DataFrame): Input dataframe.
        column (str): Column name containing text.
        top_n (int): Number of most frequent terms to return.
        
    Returns:
        pd.DataFrame: DataFrame with term counts.
    """
    # Join all text in column into one big string
    text = " ".join(df[column].astype(str).tolist())
    
    # Tokenize: lowercase words, only keep a–z characters
    tokens = re.findall(r"\b[a-zA-ZåäöÅÄÖ]+\b", text.lower())
    
    # Count terms
    counter = Counter(tokens)
    
    # Convert to DataFrame
    most_common = counter.most_common(top_n)
    return pd.DataFrame(most_common, columns=["term", "count"])



In [ ]:
bronze_data = pd.read_csv('../data/bronze/jobs.csv', index_col=0)
bronze_group = bronze_data.groupby(by=['site']).count().reset_index() 
bronze_group

,site,site_id,job_title,area,created,start_date,end_date,duration,due_date,work_location,work_type,link,raw_payload,ingestion_ts
0,A Society,95,95,95,95,85,0,0,95,95,95,95,95,95
1,Afry,78,78,78,0,0,0,0,78,74,0,78,78,78
2,Aliant,7,7,0,7,0,0,0,7,7,5,7,7,7
3,Combitech,29,29,29,0,0,0,0,29,29,0,29,29,29
4,Emagine,49,49,48,0,46,0,49,0,35,49,49,49,49
5,Ework,108,108,5,108,108,108,0,108,108,108,108,108,108
6,Nikita,20,20,0,20,0,0,0,0,0,0,20,20,20
7,Regent,26,26,26,0,26,26,0,0,26,0,26,26,26
8,Upgraded,90,90,89,0,0,0,0,90,88,38,90,90,90


In [12]:
df = most_frequent_terms(bronze_data, "job_title", top_n=100)
df.to_csv('../data/most_frequent.csv')

# Afry

In [33]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.afry_scraper import AfryScraper
s = AfryScraper()

response = s.request_status()
scraped_raw_data = s.return_raw_job_posts_data(response)
old_bronze_data = load_local_data()

new_raw_data = s.return_new_rows(new_data=scraped_raw_data, old_data=old_bronze_data)
new_bronze_data = s.parse_bronze_data(new_raw_data)
updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
unload_local_data(updated_bronze_data)

Afry > Response: 200
Afry > Nmr of scraped adds: 69
Afry > Parsing bronze data: 1


# Aliant

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.aliant_scraper import AliantScraper
s = AliantScraper()

response = s.request_status()
scraped_raw_data = s.return_raw_job_posts_data(response)
old_bronze_data = load_local_data()

new_raw_data = s.return_new_rows(new_data=scraped_raw_data, old_data=old_bronze_data)
new_bronze_data = s.parse_bronze_data(new_raw_data)
updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
unload_local_data(updated_bronze_data)


Aliant > Response: 200
Aliant > Nmr of scraped adds: 7
Aliant > Parsing bronze data: 0


# Asociety

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data

from src.scrapers.asociety_scraper import ASocietyScraper
s = ASocietyScraper()

response = s.request_status()
scraped_raw_data = s.return_raw_job_posts_data(response)
old_bronze_data = load_local_data()

new_raw_data = s.return_new_rows(new_data=scraped_raw_data, old_data=old_bronze_data)
new_bronze_data = s.parse_bronze_data(new_raw_data)
updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
unload_local_data(updated_bronze_data)

A Society > Response: 200
A Society > Nmr of scraped adds: 95
A Society > Parsing bronze data: 2


# Combitech

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.combitech_scraper import CombitechScraper
s = CombitechScraper()

response = s.request_status()
scraped_raw_data = s.return_raw_job_posts_data(response)
old_bronze_data = load_local_data()

new_raw_data = s.return_new_rows(new_data=scraped_raw_data, old_data=old_bronze_data)
new_bronze_data = s.parse_bronze_data(new_raw_data)
updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
unload_local_data(updated_bronze_data)


Combitech > Response: 200
Combitech > Nmr of scraped adds: 30
Combitech > Parsing bronze data: 0


# Emagine

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data

from src.scrapers.emagine_scraper import EmagineScraper
s = EmagineScraper()

response = s.request_status()
scraped_raw_data = s.return_raw_job_posts_data(response)
old_bronze_data = load_local_data()

new_raw_data = s.return_new_rows(new_data=scraped_raw_data, old_data=old_bronze_data)
new_bronze_data = s.parse_bronze_data(new_raw_data)
updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
unload_local_data(updated_bronze_data)


Emagine > Response: 200
Emagine > Nmr of scraped adds: 46
Emagine > Parsing bronze data: 0


# Ework

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data

from src.scrapers.ework_scraper import EworkScraper
s = EworkScraper()

response = s.request_status()
scraped_raw_data = s.return_raw_job_posts_data(response)
old_bronze_data = load_local_data()

new_raw_data = s.return_new_rows(new_data=scraped_raw_data, old_data=old_bronze_data)
new_bronze_data = s.parse_bronze_data(new_raw_data)
updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
unload_local_data(updated_bronze_data)

Ework > Response: 200
Ework > Nmr of scraped adds: 94
Ework > Parsing bronze data: 1


# Levigo 
(Inga uppdrag ute)

# Nikita 
Hemsida som renderar innehållet via JavaScript efter att själva HTML:en har laddats. I detta fall hittar man inte hittar uppdragen i Fetch/XHR i nätverksinspektionen – själva jobblistan hämtas inte via ett öppet API som returnerar JSON, utan genereras dynamiskt i webbläsaren.

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data

from src.scrapers.nikita_scraper import NikitaScraper
s = NikitaScraper()

response = s.request_status()
scraped_raw_data = s.return_raw_job_posts_data(response)
old_bronze_data = load_local_data()

new_raw_data = s.return_new_rows(new_data=scraped_raw_data, old_data=old_bronze_data)
new_bronze_data = s.parse_bronze_data(new_raw_data)
updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
unload_local_data(updated_bronze_data)

Nikita > Response: 200
Nikita > Nmr of scraped adds: 20
Nikita > Parsing bronze data: 0


# Regent

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data

from src.scrapers.regent_scraper import RegentScraper
s = RegentScraper()

response = s.request_status()
scraped_raw_data = s.return_raw_job_posts_data(response)
old_bronze_data = load_local_data()

new_raw_data = s.return_new_rows(new_data=scraped_raw_data, old_data=old_bronze_data)
new_bronze_data = s.parse_bronze_data(new_raw_data)
updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
unload_local_data(updated_bronze_data)


Regent > Response: 200
Regent > Nmr of scraped adds: 28
Regent > Parsing bronze data: 1


# Updraged 

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data

from src.scrapers.upgraded_scraper import UpgradedScraper
s = UpgradedScraper()

response = await s.request_status()
scraped_raw_data = s.return_raw_job_posts_data(response)
old_bronze_data = load_local_data()

new_raw_data = s.return_new_rows(new_data=scraped_raw_data, old_data=old_bronze_data)
new_bronze_data = s.parse_bronze_data(new_raw_data)
updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
unload_local_data(updated_bronze_data)

Upgraded > Status code: 200
Upgraded > Nmr of scraped adds: 78
Upgraded > Parsing bronze data: 1


# Run all 

In [34]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.afry_scraper import AfryScraper
from src.scrapers.aliant_scraper import AliantScraper
from src.scrapers.asociety_scraper import ASocietyScraper
from src.scrapers.combitech_scraper import CombitechScraper
from src.scrapers.emagine_scraper import EmagineScraper
from src.scrapers.ework_scraper import EworkScraper
from src.scrapers.nikita_scraper import NikitaScraper
from src.scrapers.regent_scraper import RegentScraper
from src.scrapers.upgraded_scraper import UpgradedScraper


nr_ads_pre = len(pd.read_csv('../data/bronze/jobs.csv'))

scrapers = [AfryScraper(), AliantScraper(), ASocietyScraper(), CombitechScraper(), EmagineScraper(), EworkScraper(), NikitaScraper(), RegentScraper(), UpgradedScraper()]
for s in scrapers:
    if s.site == 'Upgraded': 
        response = await s.request_status()
    else: 
        response = s.request_status()

    scraped_raw_data = s.return_raw_job_posts_data(response)
    old_bronze_data = load_local_data()

    new_raw_data = s.return_new_rows(new_data=scraped_raw_data, old_data=old_bronze_data)
    new_bronze_data = s.parse_bronze_data(new_raw_data)
    updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)
    unload_local_data(updated_bronze_data)

nr_ads_post = len(pd.read_csv('../data/bronze/jobs.csv'))
print('Total number of new added jobs:', nr_ads_post-nr_ads_pre)


Afry > Response: 200
Afry > Nmr of scraped adds: 69
Afry > Parsing bronze data: 0
Aliant > Response: 200
Aliant > Nmr of scraped adds: 7
Aliant > Parsing bronze data: 0
A Society > Response: 200
A Society > Nmr of scraped adds: 95
A Society > Parsing bronze data: 0
Combitech > Response: 200
Combitech > Nmr of scraped adds: 30
Combitech > Parsing bronze data: 0
Emagine > Response: 200
Emagine > Nmr of scraped adds: 46
Emagine > Parsing bronze data: 0
Ework > Response: 200
Ework > Nmr of scraped adds: 94
Ework > Parsing bronze data: 0
Nikita > Response: 200
Nikita > Nmr of scraped adds: 20
Nikita > Parsing bronze data: 0
Regent > Response: 200
Regent > Nmr of scraped adds: 29
Regent > Parsing bronze data: 1
Upgraded > Status code: 200
Upgraded > Nmr of scraped adds: 78
Upgraded > Parsing bronze data: 0
Total number of new added jobs: 1


# Bronze table 

In [31]:
import pandas as pd
from src.data_loader import load_local_data, unload_local_data
bronze_data = load_local_data()
print(len(bronze_data))

534
